In [13]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import json
import gzip
import ast
from textblob import TextBlob
from datetime import datetime

Apertura y lectura de archivos

In [14]:
def abrir_archivo_comprimido_1(nombre_archivo):
    '''
    Esta función abre un archivo gzip e interpreta
     interpreta cada línea como una cadena JSON y la convierte cada línea como en un objeto Python. 
    Devuelve una lista de estos diccionarios. El archivo es abierto en modo de texto 'rt'.
    '''
    with gzip.open(nombre_archivo, 'rt', encoding="UTF-8") as archivo:
        return [json.loads(line.strip()) for line in archivo]

In [15]:
def abrir_archivo_comprimido_2(nombre_archivo):
    '''
    Esta funcion abre un archivo gzip y evalúa cada línea como una expresión de Python 
    (sea JSON o no) y la convierte en un objeto Python. 
    Devuelve una lista de estos diccionarios. El archivo es abierto en modo de texto 'rt'.
    '''
    with gzip.open(nombre_archivo, 'rt', encoding="UTF-8") as archivo:
        return [ast.literal_eval(line.strip()) for line in archivo]

# DF de "user_reviews.json.gz"

In [16]:
df_review = pd.DataFrame (abrir_archivo_comprimido_2 ("Datos\\user_reviews.json.gz")) 
df_review.head(1)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."


In [17]:
df_review.shape

(25799, 3)

In [18]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [19]:
fila = 1
#fila = 120443
for i in range (0,len(df_review.columns)):
    print("Columna:", df_review.columns[i],"  // Tipo dato:", type(df_review.iloc[fila,i])," //  Informacion:", df_review.iloc[fila,i])

Columna: user_id   // Tipo dato: <class 'str'>  //  Informacion: js41637
Columna: user_url   // Tipo dato: <class 'str'>  //  Informacion: http://steamcommunity.com/id/js41637
Columna: reviews   // Tipo dato: <class 'list'>  //  Informacion: [{'funny': '', 'posted': 'Posted June 24, 2014.', 'last_edited': '', 'item_id': '251610', 'helpful': '15 of 20 people (75%) found this review helpful', 'recommend': True, 'review': 'I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it\'s title, this is easily one of my GOTYs. You don\'t get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can\'t 360 noscope your friends, but what you can do is show them up with your bad ♥♥♥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8'}, {'f

In [20]:
df_review.iloc[0,2][1]

{'funny': '',
 'posted': 'Posted July 15, 2011.',
 'last_edited': '',
 'item_id': '22200',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': "It's unique and worth a playthrough."}

In [21]:
###  Desanidar los datos de la columna 'reviews' 

# Abro el dicc que hay en 'reviews' para que cada elemento esté en una fila separada
df_review = df_review.explode('reviews').reset_index(drop=True)

# Desanido la 'reviews' en múltiples 'reviews's
df_review_des = pd.json_normalize(df_review['reviews'])

# Combino los dataframe
df_review_des= pd.concat([df_review.drop(columns=['reviews']), df_review_des], axis=1)

In [22]:
df_review_des.head(1)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...


In [23]:
df_review_des.shape

(59333, 9)

In [24]:
df_review_des.isna().sum()

user_id         0
user_url        0
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64

In [25]:
# Registros completamente vacios
print(df_review_des[df_review_des.isna().all(axis=1)].sum())
# Registros duplicados
print(df_review_des[df_review_des.duplicated()].count())

user_id        0.0
user_url       0.0
funny          0.0
posted         0.0
last_edited    0.0
item_id        0.0
helpful        0.0
recommend      0.0
review         0.0
dtype: float64
user_id        874
user_url       874
funny          874
posted         874
last_edited    874
item_id        874
helpful        874
recommend      874
review         874
dtype: int64


In [26]:
# Elimino Registros duplicados
df_review_des = df_review_des.drop_duplicates()

In [27]:
fila = 1
#fila = 120443
for i in range (0,len(df_review_des.columns)):
    print("Columna:", df_review_des.columns[i],"  // Tipo dato:", type(df_review_des.iloc[fila,i])," //  Informacion:", df_review_des.iloc[fila,i])

Columna: user_id   // Tipo dato: <class 'str'>  //  Informacion: 76561197970982479
Columna: user_url   // Tipo dato: <class 'str'>  //  Informacion: http://steamcommunity.com/profiles/76561197970982479
Columna: funny   // Tipo dato: <class 'str'>  //  Informacion: 
Columna: posted   // Tipo dato: <class 'str'>  //  Informacion: Posted July 15, 2011.
Columna: last_edited   // Tipo dato: <class 'str'>  //  Informacion: 
Columna: item_id   // Tipo dato: <class 'str'>  //  Informacion: 22200
Columna: helpful   // Tipo dato: <class 'str'>  //  Informacion: No ratings yet
Columna: recommend   // Tipo dato: <class 'bool'>  //  Informacion: True
Columna: review   // Tipo dato: <class 'str'>  //  Informacion: It's unique and worth a playthrough.


Analisis columna 'user_id

In [28]:
print("Cantidad nulos:", df_review_des['user_id'].isna().sum())
print("Tipo de dato y cantidad:", df_review_des['user_id'].apply(type).value_counts())

Cantidad nulos: 0
Tipo de dato y cantidad: <class 'str'>    58459
Name: user_id, dtype: int64


Analisis columna 'item_id'

In [29]:
print("Cantidad nulos:", df_review_des['item_id'].isna().sum())
print("Tipo de dato y cantidad:", df_review_des['item_id'].apply(type).value_counts())

Cantidad nulos: 28
Tipo de dato y cantidad: <class 'str'>      58431
<class 'float'>       28
Name: item_id, dtype: int64


In [30]:
df_review_des[df_review_des['item_id'].isna()].count()

user_id        28
user_url       28
funny           0
posted          0
last_edited     0
item_id         0
helpful         0
recommend       0
review          0
dtype: int64

In [31]:
# Como todos los registros que tienen el item_id nulo, 
# tambien tienen nulos los campos: funny,posted, last_edited, helpful, recommend,review
# Se decide eliminar todos esos registros con item_id nulo
df_review_des = df_review_des.dropna(subset=['item_id'])

Analisis columna 'recommend'

In [32]:
print("Cantidad nulos:", df_review_des['recommend'].isna().sum())
print("Tipo de dato y cantidad:", df_review_des['recommend'].apply(type).value_counts())
print("Valores unicos:" , df_review_des['recommend'].unique())

Cantidad nulos: 0
Tipo de dato y cantidad: <class 'bool'>    58431
Name: recommend, dtype: int64
Valores unicos: [True False]


Analisis columna 'Review'

In [33]:
df_review_des['review'].head()

0    Simple yet with great replayability. In my opi...
1                 It's unique and worth a playthrough.
2    Great atmosphere. The gunplay can be a bit chu...
3    I know what you think when you see this title ...
4    For a simple (it's actually not all that simpl...
Name: review, dtype: object

In [34]:
print("Cantidad nulos:", df_review_des['review'].isna().sum())
print("Tipo de dato y cantidad:", df_review_des['review'].apply(type).value_counts())

Cantidad nulos: 0
Tipo de dato y cantidad: <class 'str'>    58431
Name: review, dtype: int64


In [35]:
# Funcion para el analisis de sentimientos de la columna review.
# Se utiliza la libreria TextBlob

def analisis_sentimientos(review):
        analizador = TextBlob(review)
        if analizador.sentiment.polarity < -0.3: # Negativo
            return 0  
        elif analizador.sentiment.polarity > 0.3: # Positivo
            return 2  
        else:
            return 1  # Neutral
df_review_des['sentiment_analysis'] = df_review_des['review'].apply(analisis_sentimientos)

In [36]:
# imprimo algunos datos para corroborar el buen funcionamiento de la funcion, 
# volver y calibrar la polaridad

print("Calificacion:", df_review_des['sentiment_analysis'][1], " //  Polaridad: ", TextBlob(df_review_des['review'][1]).sentiment.polarity," //  Texto:", df_review_des['review'][1])
print("Calificacion:", df_review_des['sentiment_analysis'][0], " //  Polaridad: ", TextBlob(df_review_des['review'][0]).sentiment.polarity," //  Texto:", df_review_des['review'][0])
print("Calificacion:", df_review_des['sentiment_analysis'][7], " //  Polaridad: ", TextBlob(df_review_des['review'][7]).sentiment.polarity," //  Texto:", df_review_des['review'][7])
print("Calificacion:", df_review_des['sentiment_analysis'][9], " //  Polaridad: ", TextBlob(df_review_des['review'][9]).sentiment.polarity," //  Texto:", df_review_des['review'][9])
print("Calificacion:", df_review_des['sentiment_analysis'][10], " //  Polaridad: ", TextBlob(df_review_des['review'][10]).sentiment.polarity," //  Texto:", df_review_des['review'][10])
print("Calificacion:", df_review_des['sentiment_analysis'][16], " //  Polaridad: ", TextBlob(df_review_des['review'][16]).sentiment.polarity," //  Texto:", df_review_des['review'][16])

Calificacion: 2  //  Polaridad:  0.3375  //  Texto: It's unique and worth a playthrough.
Calificacion: 1  //  Polaridad:  0.17444444444444446  //  Texto: Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.
Calificacion: 2  //  Polaridad:  0.39999999999999997  //  Texto: "Run for fun? What the hell kind of fun is that?"
Calificacion: 0  //  Polaridad:  -0.3333333333333333  //  Texto: Random drops and random quests, with stat points.  Animation style reminiscent of the era before the Voodoo card.
Calificacion: 1  //  Polaridad:  0.19  //  Texto: Fun balance of tactics and strategy.  Potential for very rewarding battles on smaller maps.  Can become a bit of a grind on larger maps (>200 stars).
Calificacion: 0  //  Polaridad:  -0.3333333333333333  //  Texto: The ending to this game is.... ♥♥♥♥♥♥

In [37]:
df_review_des['sentiment_analysis'].value_counts()

1    43724
2    10821
0     3886
Name: sentiment_analysis, dtype: int64

Analisis columna 'posted'

In [38]:
print("Cantidad nulos:", df_review_des['posted'].isna().sum())
print("Tipo de dato y cantidad:", df_review_des['posted'].apply(type).value_counts())

Cantidad nulos: 0
Tipo de dato y cantidad: <class 'str'>    58431
Name: posted, dtype: int64


In [39]:
print(df_review_des['posted'][3])
print(df_review_des['posted'][59328])

Posted June 24, 2014.
Posted July 10.


In [40]:
def convertir_fecha(cadena_fecha):
    '''
    Esta funcion convierte una fecha tipo 'Posted %B %d, %Y.' en una fecha tipo '%Y/%m/%d' 
    Si el formato inicial de fecha es distinto, no hace nada'''
    try:
        fecha_objeto = datetime.strptime(cadena_fecha, 'Posted %B %d, %Y.')
        return fecha_objeto.strftime('%Y/%m/%d')
    except ValueError:
        return cadena_fecha

df_review_des['posted_ok'] = df_review_des['posted'].apply(convertir_fecha)

In [41]:
print("Cantidad de registros que no tienen el año y no se modificaron:",
      df_review_des['posted_ok'].str.startswith('Post').sum())
print("EJ: ",df_review_des['posted_ok'][59330])

Cantidad de registros que no tienen el año y no se modificaron: 9933
EJ:  Posted July 3.


In [42]:
# Se decide eliminar esos registros para el buen funcionamiento de la funcion.
# Tal vez se podria tomar una decision mas acertada y e imputar esos registros, 
# pero por tema de tiempo se toma esta decision.

df_review_des_ano = df_review_des.drop(df_review_des[df_review_des['posted_ok'].str.startswith('Post')].index)

In [43]:
df_review_des_ano.columns

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'review', 'sentiment_analysis', 'posted_ok'],
      dtype='object')

In [44]:
# se arma un df final, con las columnas que se necesitaran
df_review_final = df_review_des_ano[['user_id', 'user_url', 'item_id','recommend', 'posted_ok', 'sentiment_analysis']]

In [45]:
df_review_final.shape

(48498, 6)

In [46]:
df_review_final.head(1)

,user_id,user_url,item_id,recommend,posted_ok,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011/11/05,1


In [47]:
# exporto como csv
#df_review_final.to_csv("df_review_final.csv", index=False)

# Trabajamos con el DF de "users_items.json.gz"

In [48]:
df_items = pd.DataFrame (abrir_archivo_comprimido_2 ("Datos\\users_items.json.gz")) 
df_items.head(1)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [49]:
df_items.shape

(88310, 5)

In [50]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [51]:
fila = 1
for i in range (0,len(df_items.columns)):
    print("Columna:", df_items.columns[i],"  // Tipo dato:", type(df_items.iloc[fila,i])," //  Informacion:", df_items.iloc[fila,i])

Columna: user_id   // Tipo dato: <class 'str'>  //  Informacion: js41637
Columna: items_count   // Tipo dato: <class 'numpy.int64'>  //  Informacion: 888
Columna: steam_id   // Tipo dato: <class 'str'>  //  Informacion: 76561198035864385
Columna: user_url   // Tipo dato: <class 'str'>  //  Informacion: http://steamcommunity.com/id/js41637
Columna: items   // Tipo dato: <class 'list'>  //  Informacion: [{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '80', 'item_name': 'Counter-Strike: Condition Zero', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '100', 'item_name': 'Counter-Strike: Condition Zero Deleted Scenes', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_forever': 220, 'playtime_2weeks': 0}, {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime

In [52]:
df_items.iloc[0,4][1]

{'item_id': '20',
 'item_name': 'Team Fortress Classic',
 'playtime_forever': 0,
 'playtime_2weeks': 0}

In [53]:
###  Desanidar los datos de la columna 'items' 

# Abro el dicc que hay en 'items' para que cada elemento esté en una fila separada
df_items = df_items.explode('items').reset_index(drop=True)

# Desanido la 'reviews' en múltiples 'reviews's
df_items_des = pd.json_normalize(df_items['items'])

# Combino los dataframe
df_items_des= pd.concat([df_items.drop(columns=['items']), df_items_des], axis=1)

In [54]:
df_items_des.shape

(5170015, 8)

In [55]:
df_items_des.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'item_id',
       'item_name', 'playtime_forever', 'playtime_2weeks'],
      dtype='object')

In [56]:
# Registros completamente vacios
print(df_items_des[df_items_des.isna().all(axis=1)].sum())
# Registros duplicados
print(df_items_des[df_items_des.duplicated()].count())

user_id             0.0
items_count         0.0
steam_id            0.0
user_url            0.0
item_id             0.0
item_name           0.0
playtime_forever    0.0
playtime_2weeks     0.0
dtype: float64
user_id             59196
items_count         59196
steam_id            59196
user_url            59196
item_id             59104
item_name           59104
playtime_forever    59104
playtime_2weeks     59104
dtype: int64


In [57]:
# Elimino Registros duplicados
df_items_des = df_items_des.drop_duplicates()

In [58]:
# me quedo con un df mas pequeño, solo con las columnas que necesitare
df_items_des_small = df_items_des[['user_id','item_id','user_url','playtime_forever']]
df_items_des_small.head(1)

,user_id,item_id,user_url,playtime_forever
0,76561197970982479,10,http://steamcommunity.com/profiles/76561197970...,6.0


Analisis de la columna 'user_id'

In [59]:
print("Cantidad nulos:", df_items_des_small['user_id'].isna().sum())
print("Tipo de dato y cantidad:", df_items_des_small['user_id'].apply(type).value_counts())

Cantidad nulos: 0
Tipo de dato y cantidad: <class 'str'>    5110819
Name: user_id, dtype: int64


Analisis columna 'user_url'

In [60]:
print("Cantidad nulos:", df_items_des_small['user_url'].isna().sum())
print("Tipo de dato y cantidad:", df_items_des_small['user_url'].apply(type).value_counts())

Cantidad nulos: 0
Tipo de dato y cantidad: <class 'str'>    5110819
Name: user_url, dtype: int64


Analisis columna item_id

In [61]:
print("Cantidad nulos:", df_items_des_small['item_id'].isna().sum())
print("Tipo de dato y cantidad:", df_items_des_small['item_id'].apply(type).value_counts())

Cantidad nulos: 16714
Tipo de dato y cantidad: <class 'str'>      5094105
<class 'float'>      16714
Name: item_id, dtype: int64


Analisis de la columna 'playtime_forever'

In [62]:
print("Cantidad nulos:", df_items_des_small['playtime_forever'].isna().sum())
print("Tipo de dato y cantidad:", df_items_des_small['playtime_forever'].apply(type).value_counts())

Cantidad nulos: 16714
Tipo de dato y cantidad: <class 'float'>    5110819
Name: playtime_forever, dtype: int64


In [63]:
# evaluo si los registros con 'item_id' null tambien tienen 'playtime_forever' null
nulos = df_items_des_small['item_id'].isna() & df_items_des_small['playtime_forever'].isna()

df_items_des_small[nulos].count()

user_id             16714
item_id                 0
user_url            16714
playtime_forever        0
dtype: int64

In [64]:
# A los fines practicos de las funciones no me sirve tener 'user_id' o 'user_url' 
# si no tengo datos en 'item_id' o 'playtime_forever'
# se eliminan los registros con estas dos ultimas columnas nulas.

df_items_final= df_items_des_small.drop(df_items_des_small[nulos].index)

In [65]:
df_items_final.shape

(5094105, 4)

In [66]:
df_items_final.isna().sum()

user_id             0
item_id             0
user_url            0
playtime_forever    0
dtype: int64

In [67]:
df_items_final.head(1)

,user_id,item_id,user_url,playtime_forever
0,76561197970982479,10,http://steamcommunity.com/profiles/76561197970...,6.0


# Trabajamos con el DF de "steam_games.json.gz"

In [68]:
df_steam = pd.DataFrame (abrir_archivo_comprimido_1 ("Datos\\steam_games.json.gz")) 
df_steam.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561197970982479,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0


In [69]:
df_steam.shape

(120445, 19)

In [70]:
df_steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  object 
 12  id              32133 non-null  object 
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
 15  user_id         88310 non-null  object 
 16  steam_id        88310 non-null  object 
 17  items           88310 non-nul

In [71]:
#fila = 1
fila = 120443
for i in range (0,len(df_steam.columns)):
    print("Columna:", df_steam.columns[i],"  // Tipo dato:", type(df_steam.iloc[fila,i])," //  Informacion:", df_steam.iloc[fila,i])

Columna: publisher   // Tipo dato: <class 'str'>  //  Informacion: SIXNAILS
Columna: genres   // Tipo dato: <class 'list'>  //  Informacion: ['Casual', 'Indie']
Columna: app_name   // Tipo dato: <class 'str'>  //  Informacion: EXIT 2 - Directions
Columna: title   // Tipo dato: <class 'str'>  //  Informacion: EXIT 2 - Directions
Columna: url   // Tipo dato: <class 'str'>  //  Informacion: http://store.steampowered.com/app/658870/EXIT_2__Directions/
Columna: release_date   // Tipo dato: <class 'str'>  //  Informacion: 2017-09-02
Columna: tags   // Tipo dato: <class 'list'>  //  Informacion: ['Indie', 'Casual', 'Puzzle', 'Singleplayer', 'Atmospheric', 'Relaxing']
Columna: reviews_url   // Tipo dato: <class 'str'>  //  Informacion: http://steamcommunity.com/app/658870/reviews/?browsefilter=mostrecent&p=1
Columna: discount_price   // Tipo dato: <class 'numpy.float64'>  //  Informacion: nan
Columna: specs   // Tipo dato: <class 'list'>  //  Informacion: ['Single-player', 'Steam Achievements'

In [72]:
df_steam.iloc[0,17][1]

{'item_id': '20',
 'item_name': 'Team Fortress Classic',
 'playtime_forever': 0,
 'playtime_2weeks': 0}

In [73]:
df_steam.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'discount_price', 'specs', 'price',
       'early_access', 'id', 'metascore', 'developer', 'user_id', 'steam_id',
       'items', 'items_count'],
      dtype='object')

In [74]:
# luego de ver las funciones que se piden y los datos que tengo, decido quedarme con las columnas que usare

df_steam_redu = df_steam[['genres','release_date', 'price','id', 'developer']]
df_steam_redu = df_steam_redu.rename(columns={'id': 'item_id'})

In [75]:
df_steam_redu.shape

(120445, 5)

In [76]:
## cantidad de filas completamente vacias
print("Cantidad de filas vacias:", df_steam_redu.isna().all(axis=1).sum())

# elimino las filas que estan completamnete bacias
df_steam_redu = df_steam_redu.dropna(how='all')

print("Tamñano df actual:" , df_steam_redu.shape)

Cantidad de filas vacias: 88310
Tamñano df actual: (32135, 5)


In [77]:
df_steam_redu.isna().sum()

genres          3283
release_date    2067
price           1377
item_id            2
developer       3299
dtype: int64

Analisis columna 'item_id'

In [78]:
print("Cantidad nulos:", df_steam_redu['item_id'].isna().sum())
print("Cantidad duplicados:", df_steam_redu['item_id'].duplicated().sum())
print("Tipo de dato y cantidad:", df_steam_redu['item_id'].apply(type).value_counts())

Cantidad nulos: 2
Cantidad duplicados: 2
Tipo de dato y cantidad: <class 'str'>      32133
<class 'float'>        2
Name: item_id, dtype: int64


In [79]:
# Analisis de elementos duplicados
df_steam_redu[df_steam_redu['item_id'].duplicated()]

,genres,release_date,price,item_id,developer
102883,[Action],2017-10-26,59.99,612880,Machine Games
119271,"[Action, Adventure]",2012-09-07,19.99,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [80]:
df_steam_redu[df_steam_redu['item_id'] == '612880'] 

,genres,release_date,price,item_id,developer
102204,[Action],2017-10-26,59.99,612880,Machine Games
102883,[Action],2017-10-26,59.99,612880,Machine Games


In [81]:
# Dos registros iguales. Se elimina uno
df_steam_redu.drop(102204, axis=0, inplace=True)

In [82]:
df_steam_redu[df_steam_redu['item_id'].isna()]

,genres,release_date,price,item_id,developer
88384,NaN,NaN,19.99,NaN,NaN
119271,"[Action, Adventure]",2012-09-07,19.99,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [83]:
# Se elimina los registros nulo ya que el valor de 'item_id' es necesario para las funciones
df_steam_redu = df_steam_redu.dropna(subset=['item_id'])

Analisis columna 'release_date'

In [84]:
print("Cantidad nulos:", df_steam_redu['release_date'].isna().sum())
print("Cantidad duplicados:", df_steam_redu['release_date'].duplicated().sum())
print("Tipo de dato y cantidad:", df_steam_redu['release_date'].apply(type).value_counts())

Cantidad nulos: 2066
Cantidad duplicados: 28549
Tipo de dato y cantidad: <class 'str'>      30066
<class 'float'>     2066
Name: release_date, dtype: int64


In [85]:
df_steam_redu[df_steam_redu['release_date'].isna()].tail()

,genres,release_date,price,item_id,developer
120387,NaN,NaN,Free,705860,NaN
120394,"[Action, Casual, Indie]",NaN,NaN,729640,The BOMBFEST Team
120395,NaN,NaN,NaN,755540,NaN
120398,"[Action, Adventure, Indie, Early Access]",NaN,NaN,708070,rechargecomplete
120444,NaN,NaN,4.99,681550,NaN


In [86]:
# Evaluo si tengo registros que tienen todos los campos nulos exepto 'item_id', ya que no me sirve

nulos = df_steam_redu[['release_date', 'genres', 'price', 'developer']].isna().all(axis=1)
print(df_steam_redu[nulos].count())

# Se eliminan
df_steam_redu_fecha = df_steam_redu.drop(df_steam_redu[nulos].index)

genres            0
release_date      0
price             0
item_id         116
developer         0
dtype: int64


In [87]:
print("Cantidad nulos:", df_steam_redu_fecha['release_date'].isna().sum())
print("Tipo de dato y cantidad:", df_steam_redu_fecha['release_date'].apply(type).value_counts())

Cantidad nulos: 1950
Tipo de dato y cantidad: <class 'str'>      30066
<class 'float'>     1950
Name: release_date, dtype: int64


In [88]:
# remplazo los nulos por "1900-01-01"
df_steam_redu_fecha['release_date']=df_steam_redu_fecha['release_date'].fillna("1900-01-01")

In [89]:
print("Cantidad nulos:", df_steam_redu_fecha['release_date'].isna().sum())
print("Tipo de dato y cantidad:", df_steam_redu_fecha['release_date'].apply(type).value_counts())

Cantidad nulos: 0
Tipo de dato y cantidad: <class 'str'>    32016
Name: release_date, dtype: int64


In [90]:
# hay un registro que tiene cambiado el formato de fecha, se lo arregla
indice = df_steam_redu_fecha[df_steam_redu_fecha['release_date']=='15.01.2018'].index[0]

df_steam_redu_fecha.loc[indice, 'release_date'] = '2018/01/15'

In [91]:
df_steam_redu_fecha['release_date'] = pd.to_datetime(df_steam_redu_fecha['release_date'], errors='coerce')

In [92]:
print("Cantidad nulos:", df_steam_redu_fecha['release_date'].isna().sum())
print("Tipo de dato y cantidad:", df_steam_redu_fecha['release_date'].apply(type).value_counts())

Cantidad nulos: 174
Tipo de dato y cantidad: <class 'pandas._libs.tslibs.timestamps.Timestamp'>    31842
<class 'pandas._libs.tslibs.nattype.NaTType'>           174
Name: release_date, dtype: int64


In [93]:
# remplazo los nuevos nulos por "1900-01-01" y los paso a formato fecha
df_steam_redu_fecha['release_date'] = pd.to_datetime(df_steam_redu_fecha['release_date'].fillna("1900-01-01"))

In [94]:
print("Cantidad nulos:", df_steam_redu_fecha['release_date'].isna().sum())
print("Tipo de dato y cantidad:", df_steam_redu_fecha['release_date'].apply(type).value_counts())

Cantidad nulos: 0
Tipo de dato y cantidad: <class 'pandas._libs.tslibs.timestamps.Timestamp'>    32016
Name: release_date, dtype: int64


In [95]:
df_steam_redu_fecha.head(2)

,genres,release_date,price,item_id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,4.99,761140,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",2018-01-04,Free To Play,643980,Secret Level SRL


Analisis columna 'genres'

In [96]:
print("Cantidad nulos:", df_steam_redu_fecha['genres'].isna().sum())
print("Tipo de dato y cantidad:", df_steam_redu_fecha['genres'].apply(type).value_counts())

Cantidad nulos: 3166
Tipo de dato y cantidad: <class 'list'>     28850
<class 'float'>     3166
Name: genres, dtype: int64


Analisis columna 'developer'

In [97]:
print("Cantidad nulos:", df_steam_redu_fecha['developer'].isna().sum())
print("Tipo de dato y cantidad:", df_steam_redu_fecha['developer'].apply(type).value_counts())

Cantidad nulos: 3182
Tipo de dato y cantidad: <class 'str'>      28834
<class 'float'>     3182
Name: developer, dtype: int64


Analisis de la columna 'price'

In [98]:
print("Cantidad nulos:", df_steam_redu_fecha['price'].isna().sum())
print("Tipo de dato y cantidad:", df_steam_redu_fecha['price'].apply(type).value_counts())

Cantidad nulos: 1261
Tipo de dato y cantidad: <class 'float'>    30106
<class 'str'>       1910
Name: price, dtype: int64


In [99]:
# evaluo los datoas flotates
numero= df_steam_redu_fecha[df_steam_redu_fecha['price'].apply(lambda x: isinstance(x, float))]['price']
# veo si hay flotantes igual a cero 
numero[numero==float(0)]

Series([], Name: price, dtype: object)

In [100]:
# evaluo los valores free hay (misma cantidad que de str)
df_steam_redu_fecha['price'].str.startswith('Free').count()

1910

In [101]:
# remplazo todos los str que sean free por cero en formato float

df_steam_redu_fecha_price=df_steam_redu_fecha.copy()
free_str = (df_steam_redu_fecha_price['price'].str.startswith('Free')) & (df_steam_redu_fecha_price['price'].apply(type) == str)
df_steam_redu_fecha_price.loc[free_str, 'price'] = 0.0

In [102]:
print("Cantidad nulos:",df_steam_redu_fecha_price['price'].isna().sum())
print("Tipo de dato y cantidad:",df_steam_redu_fecha_price['price'].apply(type).value_counts())

Cantidad nulos: 1261
Tipo de dato y cantidad: <class 'float'>    32004
<class 'str'>         12
Name: price, dtype: int64


In [103]:
# veo cuales son los str que quedaron
df_steam_redu_fecha_price['price'][df_steam_redu_fecha_price['price'].apply(type) == str]

89279                    Play for Free!
89291                    Play for Free!
90715                       Install Now
91181     Play WARMACHINE: Tactics Demo
92142                     Install Theme
92228                       Third-party
92336                          Play Now
111044                    Play the Demo
113310              Starting at $499.00
113311              Starting at $449.00
114527                         Play Now
120148                      Third-party
Name: price, dtype: object

In [104]:
# Play for Free! lo pongo 0.0
free = df_steam_redu_fecha_price['price'] == 'Play for Free!'
df_steam_redu_fecha_price.loc[free, 'price'] = 0.0

In [105]:
# elimino el resto
eliminar = ['Play WARMACHINE: Tactics Demo', 'Install Theme', 'Third-party', 'Play Now', 'Starting at $499.00',
            'Install Now', 'Play the Demo']
df_steam_redu_fecha_price = df_steam_redu_fecha_price[~df_steam_redu_fecha_price['price'].isin(eliminar)]

In [106]:
df_steam_redu_fecha_price['price'][df_steam_redu_fecha_price['price'].apply(type) == str]

113311    Starting at $449.00
Name: price, dtype: object

In [107]:
# convierto el que me queda
free = df_steam_redu_fecha_price.index==113311
df_steam_redu_fecha_price.loc[free, 'price'] = 0.0

In [108]:
print("Cantidad nulos:", df_steam_redu_fecha_price['price'].isna().sum())
print("Tipo de dato y cantidad:", df_steam_redu_fecha_price['price'].apply(type).value_counts())

Cantidad nulos: 1261
Tipo de dato y cantidad: <class 'float'>    32007
Name: price, dtype: int64


In [117]:
df_steam_final = df_steam_redu_fecha_price.copy()
df_steam_final.head(1)

,genres,release_date,price,item_id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,4.99,761140,Kotoshiro


# FUNCIONES

# Funcion COUNTREVIEWS

def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [109]:
df_review_final.head(1)

,user_id,user_url,item_id,recommend,posted_ok,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011/11/05,1


In [110]:
# Me quedo con 'user_id','posted_ok'y'recommend' que es lo que necesito, 
# y con 'item_id', ya que 'posted_ok'y'recommend' estan a nivel de 'item_id'
# Ya revise que si hay 'posted_ok' tambien hubo reviews.

df_countreviews = df_review_final[['user_id','item_id', 'posted_ok','recommend']]
df_countreviews.head(1)

,user_id,item_id,posted_ok,recommend
0,76561197970982479,1250,2011/11/05,True


In [111]:
# Registros completamente vacios
print(df_countreviews[df_countreviews.isna().all(axis=1)].sum())
# Registros duplicados
print(df_countreviews[df_countreviews.duplicated()].count())

user_id      0.0
item_id      0.0
posted_ok    0.0
recommend    0.0
dtype: float64
user_id      0
item_id      0
posted_ok    0
recommend    0
dtype: int64


In [112]:
# Ahora que ya corrobore que no hay nulos ni duplicados, puedo elimnar la coliimna item_id
df_countreviews = df_countreviews.drop(columns=['item_id','user_id'])

In [256]:
#cambio el formato de la fecha
df_countreviews['posted_ok'] = df_countreviews['posted_ok'].str.replace('/', '-')

In [257]:
df_countreviews.shape

(48498, 2)

In [258]:
df_countreviews.head(1)

,posted_ok,recommend
0,2011-11-05,True


In [259]:
#df_countreviews.to_json("DataSets/df_countreviews.json", orient="records", lines=True)

# Funcion GENRES

def genre( género : str ): Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [118]:
df_item_genres = df_steam_final[['genres', 'item_id']]

In [119]:
# descompongo la lista de generos
df_item_genres = df_item_genres.explode('genres')

In [120]:
df_item_genres['genres'].isna().sum()

3165

In [121]:
# Elimino los registros donde 'genres' es nulo
df_item_genres = df_item_genres.dropna(subset=['genres'])

In [122]:
# Veo que hay adentro de 'genres'
df_item_genres['genres'].unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [123]:
# Elimino todo lo que no es genero de pelicula

eliminar = ['Free to Play','Early Access','Animation &amp; Modeling','Video Production','Utilities',
            'Web Publishing','Education','Software Training','Design &amp; Illustration',
            'Audio Production','Photo Editing','Accounting']

condicion = ~df_item_genres['genres'].isin(eliminar)
df_item_genres = df_item_genres[condicion]

In [124]:
df_item_genres.shape

(66268, 2)

In [275]:
df_item_genres.head(1)

,genres,item_id
88310,Action,761140


In [126]:
# creo un df con 'item' y 'playtime_forever'
df_item_playtime = df_items_final[['item_id','playtime_forever']]

In [127]:
df_item_playtime['playtime_forever'].isna().sum()

0

In [128]:
df_item_playtime.shape

(5094105, 2)

In [129]:
df_item_playtime.head(1)

,item_id,playtime_forever
0,10,6.0


In [130]:
# junto las dos tablas a travez del 'item_id'
df_genre = pd.merge(df_item_genres,df_item_playtime, on='item_id')

In [131]:
df_genre = df_genre.drop(columns=['item_id'])

In [132]:
df_genre.head(1)

,genres,playtime_forever
0,Action,5.0


In [133]:
# agrupo por 'item_id' y sumo  los 'price'
df_genre = df_genre.groupby('genres')['playtime_forever'].sum().reset_index()
df_genre.head(3)

,genres,playtime_forever
0,Action,3.075100e+09
1,Adventure,8.987636e+08
2,Casual,2.492682e+08


In [143]:
# ordeno segun los valores de la columna 'playtime_forever'
df_genre_orden = df_genre.sort_values(by='playtime_forever', ascending=False)

In [144]:
# agrego una columna para enumerar el orden
df_genre_orden['ranking'] = range(1, 11)

In [145]:
df_genre_orden.head(3)

,genres,playtime_forever,ranking
0,Action,3.075100e+09,1
3,Indie,1.475435e+09,2
5,RPG,1.027928e+09,3


In [146]:
df_genre_ranking = df_genre_orden[['genres','ranking']]

In [147]:
# convierto a misnucula para evitar errores
df_genre_ranking['genres'] = df_genre_ranking['genres'].str.lower()

C:\Users\peros\AppData\Local\Temp\ipykernel_13516\2575041377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre_ranking['genres'] = df_genre_ranking['genres'].str.lower()


In [148]:
df_genre_ranking.head(1)

,genres,ranking
0,action,1


In [149]:
#df_genre_ranking.to_json("DataSets/df_genre.json", orient="records", lines=True)

# Funcion USERFORGENRE

def userforgenre( género : str ): Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [150]:
df_item_genres.head(1)

,genres,item_id
88310,Action,761140


In [151]:
df_items_final.head(1)

,user_id,item_id,user_url,playtime_forever
0,76561197970982479,10,http://steamcommunity.com/profiles/76561197970...,6.0


In [152]:
# junto las dos tablas a travez del 'item_id'
df_userforgenre_inicial = pd.merge(df_item_genres,df_items_final, on='item_id')

In [153]:
df_userforgenre_inicial = df_userforgenre_inicial.drop(columns=['item_id','user_url']) # elimino columnas

In [154]:
df_userforgenre_inicial.head(2)

,genres,user_id,playtime_forever
0,Action,UTNerd24,5.0
1,Action,I_DID_911_JUST_SAYING,0.0


In [155]:
def ranking_genero (genero):
    ''' Esta funcion recibe un genero y devulve un df con un Top 5 de usuarios 
    con más horas de juego (sum de playtime_forever) en el género dado, su user_url y user_id '''

    df_final = pd.DataFrame()
    df_final = df_userforgenre_inicial[df_userforgenre_inicial['genres'] == genero] # hago un df por genero
    df_final = df_final.drop(columns=['genres']) # elimino columna genero
    df_final = df_final.groupby('user_id')['playtime_forever'].sum().reset_index() # agrupo por 'user_id' y sumo 'play'
    df_final = df_final.sort_values(by='playtime_forever', ascending=False) # ordeno segun los valores de la columna 'playtime_forever'
    df_final = df_final.head(5).copy() # me quedo con los 5 primeros
    df_final['ranking'] = range(1, 6)# agrego una columna para enumerar el orden
    df_final['genres'] = genero # agrego la columna de genero
    df_final['user_url'] = None # genero una columna de url

    # para ponerle el 'user_url' a cada 'user_id'
    for elemento in df_final['user_id']:
        filtrado = df_items_final.loc[df_items_final['user_id'] == elemento, 'user_url']
        df_final.loc[df_final['user_id'] == elemento, 'user_url'] = filtrado.values[0]
    return df_final

In [156]:
df_userforgenre_inicial['genres'].unique()

array(['Action', 'Indie', 'Racing', 'Strategy', 'RPG', 'Simulation',
       'Casual', 'Adventure', 'Sports', 'Massively Multiplayer'],
      dtype=object)

In [158]:
genero = 'Action'
df_userforgenre_action = ranking_genero (genero)
df_userforgenre_action.head(1)

,user_id,playtime_forever,ranking,genres,user_url
50657,Sp3ctre,1699307.0,1,Action,http://steamcommunity.com/id/Sp3ctre


In [159]:
genero = 'Indie'
df_userforgenre_indie = ranking_genero (genero)
df_userforgenre_indie.head(3)

,user_id,playtime_forever,ranking,genres,user_url
46547,REBAS_AS_F-T,2402994.0,1,Indie,http://steamcommunity.com/id/REBAS_AS_F-T
56093,jimmynoe,1139510.0,2,Indie,http://steamcommunity.com/id/jimmynoe
61218,shinomegami,1111013.0,3,Indie,http://steamcommunity.com/id/shinomegami


In [160]:
genero = 'Racing'
df_userforgenre_racing = ranking_genero (genero)
df_userforgenre_racing.head(3)

,user_id,playtime_forever,ranking,genres,user_url
16798,DownSyndromeKid,743445.0,1,Racing,http://steamcommunity.com/id/DownSyndromeKid
1313,76561197995725436,193206.0,2,Racing,http://steamcommunity.com/profiles/76561197995...
5155,76561198045342815,136459.0,3,Racing,http://steamcommunity.com/profiles/76561198045...


In [161]:
genero = 'Strategy'
df_userforgenre_strategy = ranking_genero (genero)
df_userforgenre_strategy.head(3)

,user_id,playtime_forever,ranking,genres,user_url
52337,shinomegami,1141546.0,1,Strategy,http://steamcommunity.com/id/shinomegami
51678,rinco69,613757.0,2,Strategy,http://steamcommunity.com/id/rinco69
15997,76561198063368177,612812.0,3,Strategy,http://steamcommunity.com/profiles/76561198063...


In [162]:
genero = 'RPG'
df_userforgenre_rpg = ranking_genero (genero)
df_userforgenre_rpg.head(3)

,user_id,playtime_forever,ranking,genres,user_url
56592,shinomegami,1060592.0,1,RPG,http://steamcommunity.com/id/shinomegami
58179,tobscene,906936.0,2,RPG,http://steamcommunity.com/id/tobscene
42585,REBAS_AS_F-T,886204.0,3,RPG,http://steamcommunity.com/id/REBAS_AS_F-T


In [163]:
genero = 'Simulation'
df_userforgenre_simulation = ranking_genero (genero)
df_userforgenre_simulation.head(3)

,user_id,playtime_forever,ranking,genres,user_url
50396,jimmynoe,1062130.0,1,Simulation,http://steamcommunity.com/id/jimmynoe
36818,Evilutional,684723.0,2,Simulation,http://steamcommunity.com/id/Evilutional
41394,REBAS_AS_F-T,676540.0,3,Simulation,http://steamcommunity.com/id/REBAS_AS_F-T


In [164]:
genero = 'Casual'
df_userforgenre_casual = ranking_genero (genero)
df_userforgenre_casual.head(3)

,user_id,playtime_forever,ranking,genres,user_url
39758,REBAS_AS_F-T,1224933.0,1,Casual,http://steamcommunity.com/id/REBAS_AS_F-T
47963,idonothack,781178.0,2,Casual,http://steamcommunity.com/id/idonothack
53287,shinomegami,671493.0,3,Casual,http://steamcommunity.com/id/shinomegami


In [165]:
genero = 'Adventure'
df_userforgenre_adventure = ranking_genero (genero)
df_userforgenre_adventure.head(3)

,user_id,playtime_forever,ranking,genres,user_url
44738,REBAS_AS_F-T,2191551.0,1,Adventure,http://steamcommunity.com/id/REBAS_AS_F-T
59140,shinomegami,767160.0,2,Adventure,http://steamcommunity.com/id/shinomegami
40006,Evilutional,662103.0,3,Adventure,http://steamcommunity.com/id/Evilutional


In [166]:
genero ='Sports'
df_userforgenre_sports = ranking_genero (genero)
df_userforgenre_sports.head(3)

,user_id,playtime_forever,ranking,genres,user_url
1476,76561197995725436,193206.0,1,Sports,http://steamcommunity.com/profiles/76561197995...
10489,76561198065902279,126572.0,2,Sports,http://steamcommunity.com/profiles/76561198065...
29705,express157,122179.0,3,Sports,http://steamcommunity.com/id/express157


In [167]:
genero = 'Massively Multiplayer'
df_userforgenre_massi = ranking_genero (genero)
df_userforgenre_massi.head(3)

,user_id,playtime_forever,ranking,genres,user_url
35215,Evilutional,688260.0,1,Massively Multiplayer,http://steamcommunity.com/id/Evilutional
52740,shinomegami,638699.0,2,Massively Multiplayer,http://steamcommunity.com/id/shinomegami
6887,76561198041627248,457103.0,3,Massively Multiplayer,http://steamcommunity.com/profiles/76561198041...


In [168]:
# concateno todos los ranking
df_userforgenre = pd.concat([df_userforgenre_massi,df_userforgenre_action, df_userforgenre_adventure,
                             df_userforgenre_casual, df_userforgenre_indie, df_userforgenre_strategy,
                             df_userforgenre_racing, df_userforgenre_rpg, df_userforgenre_sports,
                             df_userforgenre_simulation]).reset_index(drop=True)

In [169]:
df_userforgenre.head(4)

,user_id,playtime_forever,ranking,genres,user_url
0,Evilutional,688260.0,1,Massively Multiplayer,http://steamcommunity.com/id/Evilutional
1,shinomegami,638699.0,2,Massively Multiplayer,http://steamcommunity.com/id/shinomegami
2,76561198041627248,457103.0,3,Massively Multiplayer,http://steamcommunity.com/profiles/76561198041...
3,76561198050414588,454132.0,4,Massively Multiplayer,http://steamcommunity.com/profiles/76561198050...


In [171]:
df_userforgenre['genres'] = df_userforgenre['genres'].str.lower()

In [172]:
#df_userforgenre.to_json("DataSets/df_userforgenre.json", orient="records", lines=True)

# Funcion DEVELOPER

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:

In [173]:
df_steam_final.head(1)

,genres,release_date,price,item_id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,4.99,761140,Kotoshiro


In [174]:
# me quedo con la columnas necesarias
df_steam_developer = df_steam_final[['release_date', 'price', 'item_id', 'developer']]

In [175]:
df_steam_developer.head(5)

,release_date,price,item_id,developer
88310,2018-01-04,4.99,761140,Kotoshiro
88311,2018-01-04,0.0,643980,Secret Level SRL
88312,2017-07-24,0.0,670290,Poolians.com
88313,2017-12-07,0.99,767400,彼岸领域
88314,1900-01-01,2.99,773570,NaN


In [176]:
# Registros completamente vacios
print(df_steam_developer [df_steam_developer .isna().all(axis=1)].count())
# Registros duplicados
print(df_steam_developer [df_steam_developer .duplicated()].count())

release_date    0
price           0
item_id         0
developer       0
dtype: int64
release_date    0
price           0
item_id         0
developer       0
dtype: int64


In [177]:
df_steam_developer.shape

(32007, 4)

In [178]:
# registros que price o devolper son nan o que release_date es 1900 (no hay ningun registro nulo en item_id)
# los elimino porque tendre la infomacion incompleta
eliminar = df_steam_developer[(df_steam_developer['release_date'] == datetime(1900, 1, 1)) | (df_steam_developer['price'].isnull() | df_steam_developer['developer'].isnull())]
df_steam_developer_elim = df_steam_developer[~df_steam_developer.index.isin(eliminar.index)]

In [179]:
df_steam_developer_elim.shape

(27584, 4)

In [180]:
# creo una columna solo con el año
df_steam_developer_elim['año'] = df_steam_developer_elim['release_date'].dt.year

C:\Users\peros\AppData\Local\Temp\ipykernel_13516\2900269083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_developer_elim['año'] = df_steam_developer_elim['release_date'].dt.year


In [181]:
# elimino columna 'release_date'
df_steam_developer_elim = df_steam_developer_elim.drop(columns=['release_date'])

In [182]:
df_steam_developer_elim.head(2)

,price,item_id,developer,año
88310,4.99,761140,Kotoshiro,2018
88311,0.0,643980,Secret Level SRL,2018


In [183]:
# convierto los developers a minuscula
df_steam_developer_elim ['developer'] = df_steam_developer_elim ['developer'].str.lower()

In [184]:
# veo cuantos Nan hay en 'developer'
df_steam_developer_elim [df_steam_developer_elim ['developer'].isna()].shape

(0, 4)

In [185]:
# veo cuantos Nan hay en 'price'
df_steam_developer_elim [df_steam_developer_elim ['price'].isna()].shape

(0, 4)

In [186]:
#df_steam_developer_elim.to_json("DataSets/df_developer.json", orient="records", lines=True)

# Funcion SENTIMENT_ANALYSIS

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

                    Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [199]:
df_steam_final.head(1)

,genres,release_date,price,item_id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,4.99,761140,Kotoshiro


In [200]:
df_item_date = df_steam_final[['release_date', 'item_id']]

In [201]:
df_review_final.head(1)

,user_id,user_url,item_id,recommend,posted_ok,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011/11/05,1


In [202]:
df_item_sentiment = df_review_final[['item_id', 'sentiment_analysis']]

In [203]:
df_sentiment_inicial = pd.merge(df_item_date, df_item_sentiment, on='item_id')

In [204]:
df_sentiment_inicial = df_sentiment_inicial.drop(columns=['item_id'])

In [205]:
# creo una columna solo con el año
df_sentiment_inicial['año'] = df_sentiment_inicial['release_date'].dt.year
# elimino columna 'release_date'
df_sentiment_inicial = df_sentiment_inicial.drop(columns=['release_date'])

In [206]:
df_sentiment_inicial['año'].unique()

array([1998, 2006, 2005, 2000, 2007, 1995, 1994, 2001, 2004, 2003, 2008,
       1997, 1993, 2009, 2002, 2010, 2011, 2013, 2012, 1900, 2015, 1996,
       2014, 1992, 2016, 1989, 2017, 1999, 1991, 1990], dtype=int64)

In [207]:
# creo un dataframe para cargar los años y las cantidades de cada sentimiento
df_sentiment = pd.DataFrame(columns=['ano','cantidad_0','cantidad_1','cantidad_2'])
df_sentiment['ano']=df_sentiment_inicial['año'].unique()

In [208]:
# remplazo cada nan por la cantidad de sentimientos 0, 1 o 2 que hay para cada año
for elemento in df_sentiment['ano']:
    conteo_sentiment = df_sentiment_inicial['sentiment_analysis'][df_sentiment_inicial['año']==elemento].value_counts()

    # si no hay elemento en ese año, le pongo 0
    df_sentiment.loc[df_sentiment['ano'] == elemento, 'cantidad_0'] = conteo_sentiment.get(0,0) 
    df_sentiment.loc[df_sentiment['ano'] == elemento, 'cantidad_1'] = conteo_sentiment.get(1,0)
    df_sentiment.loc[df_sentiment['ano'] == elemento, 'cantidad_2'] = conteo_sentiment.get(2,0) 

In [210]:
df_sentiment.head(2)

,ano,cantidad_0,cantidad_1,cantidad_2
0,1998,2,66,14
1,2006,122,1334,379


In [211]:
#df_sentiment.to_json("DataSets/df_sentiment.json", orient="records", lines=True)

# Funcion USERDATA

def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [212]:
df_review_final.head(1)

,user_id,user_url,item_id,recommend,posted_ok,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011/11/05,1


In [260]:
# Me quedo con 'user_id' y 'recommend' que es lo que necesito, 
# y con 'item_id', ya que 'recommend' estan a nivel de 'item_id'

df_user_recomm = df_review_final[['user_id', 'item_id','recommend']]
df_user_recomm.head(1)

,user_id,item_id,recommend
0,76561197970982479,1250,True


In [261]:
# Registros completamente vacios
print(df_user_recomm[df_user_recomm.isna().all(axis=1)].count())
# Registros duplicados. 
print(df_user_recomm[df_user_recomm.duplicated()].count())

user_id      0
item_id      0
recommend    0
dtype: int64
user_id      0
item_id      0
recommend    0
dtype: int64


In [262]:
# Ahora que ya corrobore que no hay nulos ni duplicados, puedo elimnar la colummna item_id
df_user_recomm = df_user_recomm.drop(columns=['item_id'])

In [263]:
print("Cantidad nulos:", df_user_recomm['recommend'].isna().sum())
print("Tipo de dato y cantidad:", df_user_recomm['recommend'].apply(type).value_counts())

print("Cantidad nulos:", df_user_recomm['user_id'].isna().sum())
print("Tipo de dato y cantidad:", df_user_recomm['user_id'].apply(type).value_counts())

Cantidad nulos: 0
Tipo de dato y cantidad: <class 'bool'>    48498
Name: recommend, dtype: int64
Cantidad nulos: 0
Tipo de dato y cantidad: <class 'str'>    48498
Name: user_id, dtype: int64


In [264]:
# Agrupo por 'user_id' y cuento cuantas recomendaciones True o False hay
# Coloco cero si no hay ese tipo de recomendación

df_porc_recomm = df_user_recomm.groupby(['user_id', 'recommend'])['recommend'].count().unstack(fill_value=0).reset_index()

In [265]:
df_porc_recomm.head(2)

recommend,user_id,False,True
0,--000--,0,1
1,--ace--,0,2


In [266]:
# Conviero los valores a porcentaje

df_porc_recomm[True] = round((df_porc_recomm[True] / (df_porc_recomm[True] + df_porc_recomm[False])) * 100,1)

# Renombro la columna
df_porc_recomm = df_porc_recomm.rename(columns={True: 'Recomendacion'})

In [268]:
df_porc_recomm = df_porc_recomm.drop(columns={False})

In [269]:
df_porc_recomm.tail(2)

recommend,user_id,Recomendacion
22793,zyr0n1c,100.0
22794,zzoptimuszz,100.0


In [270]:
#df_porc_recomm.to_json("DataSets/df_porc_recomm.json", orient="records", lines=True)

In [225]:
df_steam_final.head(1)

,genres,release_date,price,item_id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,4.99,761140,Kotoshiro


In [227]:
df_item_price = df_steam_final[['item_id','price']]
df_item_price.head(1)

,item_id,price
88310,761140,4.99


In [229]:
# Registros completamente vacios
print(df_item_price[df_item_price.isna().all(axis=1)].count())
# Registros duplicados
print(df_item_price[df_item_price.duplicated()].count())

item_id    0
price      0
dtype: int64
item_id    0
price      0
dtype: int64


In [230]:
print("Cantidad nulos:", df_item_price['item_id'].isna().sum())
print("Tipo de dato y cantidad:", df_item_price['item_id'].apply(type).value_counts())
print("Cantidad nulos:", df_item_price['price'].isna().sum())
print("Tipo de dato y cantidad:", df_item_price['price'].apply(type).value_counts())

Cantidad nulos: 0
Tipo de dato y cantidad: <class 'str'>    32007
Name: item_id, dtype: int64
Cantidad nulos: 1261
Tipo de dato y cantidad: <class 'float'>    32007
Name: price, dtype: int64


In [231]:
# Lo ideal seria realizar una imputacion de los nulos, utilizando otros registros. 
# Por temas de tiempo Elimino los registros con datos nulo. 
df_item_price = df_item_price.dropna(subset=['price'])

In [238]:
df_item_price.head(1)

,item_id,price
88310,761140,4.99


In [233]:
df_items_final.head(1)

,user_id,item_id,user_url,playtime_forever
0,76561197970982479,10,http://steamcommunity.com/profiles/76561197970...,6.0


In [234]:
df_item_user = df_items_final[['user_id','item_id']]
df_item_user.head(3)

,user_id,item_id
0,76561197970982479,10
1,76561197970982479,20
2,76561197970982479,30


In [235]:
# Registros completamente vacios
print(df_item_user[df_item_user.isna().all(axis=1)].count())
# Registros duplicados
print(df_item_user[df_item_user.duplicated()].count())

user_id    0
item_id    0
dtype: int64
user_id    23
item_id    23
dtype: int64


In [236]:
# Elimino Registros duplicados, ya que cada item es jugado por un usuario
df_item_user = df_item_user.drop_duplicates()

In [237]:
# agrupo por 'user_id' y cuento los 'item_id'
df_item_user_group = df_item_user[['user_id', 'item_id']].groupby('user_id').count().reset_index()
df_item_user_group.head(1)

,user_id,item_id
0,--000--,58


In [271]:
# Renombro la columna
df_item_user_group = df_item_user_group.rename(columns={'item_id': 'Cantidad_item_id'})

In [272]:
#df_item_user_group.to_json("DataSets/df_item_user_group.json", orient="records", lines=True)

In [240]:
# Convino los registros utilizando 'item_id'
df_user_price = pd.merge(df_item_price, df_item_user, on='item_id')
df_user_price.head(3)

,item_id,price,user_id
0,282010,9.99,UTNerd24
1,282010,9.99,I_DID_911_JUST_SAYING
2,282010,9.99,76561197962104795


In [241]:
# Registros completamente vacios
print(df_user_price[df_user_price.isna().all(axis=1)].count())
# Registros duplicados
print(df_user_price[df_user_price.duplicated()].count())

item_id    0
price      0
user_id    0
dtype: int64
item_id    0
price      0
user_id    0
dtype: int64


In [242]:
# Ahora que ya corrobore que no hay nulos ni duplicados, puedo elimnar la colummna item_id
df_user_price = df_user_price.drop(columns=['item_id'])

In [244]:
# agrupo por 'user_id' y sumo los 'price'
df_user_price = df_user_price.groupby('user_id')['price'].sum().reset_index()
df_user_price.head(1)

,user_id,price
0,--000--,402.77


In [254]:
df_user_price['price'] = round(df_user_price['price'],2)

In [255]:
#df_user_price.to_json("DataSets/df_user_price.json", orient="records", lines=True)

# Funcion MODELO DE RECOMENDACION

In [289]:
df_item_genres.head(1)

,genres,item_id
88310,Action,761140


In [290]:
df_steam_developer_elim.head(1)

,price,item_id,developer,año
88310,4.99,761140,kotoshiro,2018


In [291]:
# Armo el df con las columnas que voy a usar para el modelo de recomendacion
df_item_genre_año = pd.merge(df_item_genres, df_steam_developer_elim[['item_id', 'año']], on='item_id')
df_item_genre_año.head(2)

,genres,item_id,año
0,Action,761140,2018
1,Casual,761140,2018


In [292]:
# Preparo el DF para el modelo de recomendacion. 
# Nececito tener un df con items en las filas y generos y años en las columnas con valores binarios

# Con get dummies obtengo columnas binarias a partir de 'genres' y 'año'
df_genres_binario = pd.get_dummies(df_item_genre_año['genres'])

df_año_binario = pd.get_dummies(df_item_genre_año['año'])

In [294]:
# Concateno este nuevo df con el anterior para que me quede por 'item_id'
df_item_genre_año_bi = pd.concat([df_item_genre_año['item_id'], df_genres_binario, df_año_binario], axis=1)

In [296]:
# Tengo que juntar las filas de un mismo item_id

# Agrupo por item_id y sumo los valores de las columnas 
df_item_genre_año_bi = df_item_genre_año_bi.groupby('item_id').sum()

# Y si hay algun valor mayor a 1 le asigno 1
df_item_genre_año_bi[df_item_genre_año_bi > 1] = 1

In [297]:
df_item_genre_año_bi.head(5)

,Action,Adventure,Casual,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2021
item_id,,,,,,,,,,,,,,,,,,,,,
10,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1002,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10090,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10100,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10110,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [300]:
df_item_genre_año_bi.index

Index(['10', '1002', '10090', '10100', '10110', '10120', '10130', '10140',
       '10150', '10180',
       ...
       '99610', '99611', '99612', '99614', '9970', '99700', '9980', '99900',
       '99910', '9992'],
      dtype='object', name='item_id', length=26681)

In [303]:
#df_item_genre_año_bi.to_json("DataSets/df_modelo.json")